#Linear regression using PyTorch built-ins


We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [54]:
import torch

In [55]:
import numpy as np

In [56]:
import torch.nn as nn

In [57]:
#Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [58]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [59]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches.



#Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [60]:
from torch.utils.data import TensorDataset

In [61]:
#define dataset
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [62]:
from torch.utils.data import DataLoader

In [63]:
#define data loader
batch_size = 3
train_dl= DataLoader(train_ds,batch_size,shuffle=True)

We can use the data loader in a for loop. Let's look at an example.

In [64]:
for xb,yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[101.,  44.,  37.],
        [103.,  43.,  36.],
        [102.,  43.,  37.]])
tensor([[21., 38.],
        [20., 38.],
        [22., 37.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

#nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [65]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5691,  0.0758,  0.5460],
        [ 0.1605,  0.3565, -0.5010]], requires_grad=True)
Parameter containing:
tensor([ 0.4753, -0.3646], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [66]:
# parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.5691,  0.0758,  0.5460],
         [ 0.1605,  0.3565, -0.5010]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4753, -0.3646], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [67]:
#generate predictions
preds = model(inputs)
preds

tensor([[-12.5080,  13.6950],
        [ -9.6930,  13.5497],
        [ -7.2041,  32.3102],
        [-34.1066,  12.8014],
        [  6.7088,   9.8638],
        [-13.1529,  13.4991],
        [ -9.2229,  12.6923],
        [ -7.2272,  31.9697],
        [-33.4617,  12.9974],
        [  7.8238,   9.2023],
        [-12.0379,  12.8376],
        [-10.3379,  13.3538],
        [ -7.6742,  33.1676],
        [-35.2216,  13.4629],
        [  7.3537,  10.0597]], grad_fn=<AddmmBackward>)

Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [68]:
# import nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [69]:
#define the loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [70]:
loss = loss_fn(model(inputs),targets)

In [71]:
print(loss)

tensor(7454.4858, grad_fn=<MseLossBackward>)


#Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group

In [72]:
# define optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)


Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

#Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [73]:
# utility function to train the model
def fit(num_epochs,model,loss_fn,opt,train_dl):
  #repeat for given number of epoches
  for epoch in range(num_epochs):

    #train with batches of data
    for xb,yb in train_dl:
      #1 generate predictions
      pred = model(xb)
      
      #2 calcuate loss
      loss = loss_fn(pred, yb)

      #3 computer gradients
      loss.backward()

      #4 update parameters using gradients
      opt.step()

      #5 reset the gradients to zero
      opt.zero_grad()
    #print the progress
    if(epoch +1) % 10 == 0:
      print('Epoch [{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,loss.item()))


Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [74]:
fit(200,model,loss_fn,opt,train_dl)


Epoch [10/200],Loss:204.0828
Epoch [20/200],Loss:102.1040
Epoch [30/200],Loss:62.0726
Epoch [40/200],Loss:52.2526
Epoch [50/200],Loss:36.5705
Epoch [60/200],Loss:35.9019
Epoch [70/200],Loss:65.5519
Epoch [80/200],Loss:23.4633
Epoch [90/200],Loss:27.0687
Epoch [100/200],Loss:38.2539
Epoch [110/200],Loss:18.2844
Epoch [120/200],Loss:23.8462
Epoch [130/200],Loss:2.0479
Epoch [140/200],Loss:26.6719
Epoch [150/200],Loss:16.8459
Epoch [160/200],Loss:29.4819
Epoch [170/200],Loss:14.7331
Epoch [180/200],Loss:17.7844
Epoch [190/200],Loss:14.9084
Epoch [200/200],Loss:19.0774


Let's generate predictions using our model and verify that they're close to our targets.



In [75]:
# generate predictions

In [76]:
preds = model(inputs)

In [77]:
preds

tensor([[ 57.0313,  70.5474],
        [ 82.4761,  97.4066],
        [117.3355, 138.8184],
        [ 20.6394,  39.3422],
        [102.8105, 112.1164],
        [ 55.7909,  69.4099],
        [ 82.4097,  96.9516],
        [117.6676, 139.1222],
        [ 21.8798,  40.4797],
        [103.9845, 112.7989],
        [ 56.9649,  70.0925],
        [ 81.2357,  96.2691],
        [117.4019, 139.2733],
        [ 19.4654,  38.6597],
        [104.0509, 113.2538]], grad_fn=<AddmmBackward>)

In [78]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [79]:
model(torch.tensor([[75,63,44.]]))

tensor([[53.6647, 66.8694]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

